[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/utkarshp1161/Active-learning-in-microscopy/blob/main/notebooks/Beacon-combi-library.ipynb)
# Notebook illustrating novelty search - BEACON method - (2dim input space) to (1 or 2 dim target space)

## Here we apply it to combi library exploration on the grid data - We benchmark the exploration on various acquisiton functions.
- credits
    - original paper - [BEACON: A Bayesian Optimization Strategy for Novelty Search in Expensive Black-Box Systems](https://arxiv.org/abs/2406.03616)
        - Wei-Ting Tang, Ankush Chakrabarty, Joel A. Paulson
    - Adapted code by [Utkarsh Pratiush](https://github.com/utkarshp1161)

    - Data by [Richard Liu](https://github.com/RichardLiuCoding)

# Note: Recommended to take a GPU instance


## 1. Install stuff

In [ ]:
# botorch - gpytorch etc
#install
!pip install botorch==0.12.0
!pip install gpytorch==1.13

In [ ]:
import torch
import gpytorch
from botorch.models import SingleTaskGP, ModelListGP
from gpytorch.kernels import RBFKernel, ScaleKernel
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import AcquisitionFunction
from botorch.utils.transforms import t_batch_mode_transform
import numpy as np
import matplotlib.pyplot as plt
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.models.transforms.outcome import Standardize
from botorch.models.model import Model
from typing import Optional
from botorch.acquisition.objective import PosteriorTransform
from torch import Tensor

## 2. Write the Thompson sampler which is core for sampling in beacon

In [ ]:
## Thompson sampling on GPU - credits - REPO: https://github.com/PaulsonLab/BEACON

import numpy as np
import torch
from math import pi
from typing import Any, Tuple
from botorch.models.model import Model
from botorch.models import SingleTaskGP
from botorch.posteriors import Posterior
from botorch.models.transforms import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood

class EfficientThompsonSampler():
    def __init__(self, model, num_of_multistarts=1, num_of_bases=1024, num_of_samples=1):
        '''
        Implementation of 'Efficiently Sampling From Gaussian Process Posteriors' by Wilson et al. (2020). It allows
        us to create approximate samples of the GP posterior, which we can optimise using gradient methods. We do this
        to generate candidates using Thompson Sampling. Link to the paper: https://arxiv.org/pdf/2002.09309.pdf .
        
        GPU-enabled version.
        '''
        # GP model
        self.model = model
        
        # Determine device from model
        if hasattr(model, 'train_inputs'):
            if isinstance(model.train_inputs, tuple):
                self.device = model.train_inputs[0].device
            else:
                self.device = model.train_inputs.device
        else:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # inputs
        if type(self.model.train_x) == torch.Tensor:
            self.train_x = self.model.train_x.to(self.device)
        else:
            self.train_x = torch.tensor(self.model.train_x, device=self.device)
        
        self.x_dim = torch.tensor(self.train_x.shape[1], device=self.device)
        self.train_y = self.model.train_y.to(self.device)
        self.num_of_train_inputs = self.model.train_x.shape[0]
        
        # scaled outputs
        self.scaled_train_y = self.model.outcome_transform(self.train_y)[0].to(self.device)
        
        # thompson sampling parameters
        self.num_of_multistarts = num_of_multistarts
        self.num_of_bases = num_of_bases
        self.num_of_samples = num_of_samples
        
        # optimisation parameters
        self.learning_rate = 0.01
        self.num_of_epochs = 10 * self.x_dim
        
        # obtain the kernel parameters
        self.sigma = self.model.likelihood.noise[0].item()  # assumes fixed noise value
        self.lengthscale = self.model.covar_module.base_kernel.lengthscale.detach().float().to(self.device)
        self.outputscale = self.model.covar_module.outputscale.item()
        
        # obtain the kernel
        self.kernel = self.model.covar_module
        
        # define the Knn matrix
        with torch.no_grad():
            self.Knn = self.kernel(self.train_x)
            self.Knn = self.Knn.evaluate().to(self.device)
            # precalculate matrix inverse
            self.inv_mat = torch.inverse(
                self.Knn + self.sigma * torch.eye(self.num_of_train_inputs, device=self.device)
            )

        self.create_fourier_bases()
        self.calculate_phi()

    def create_fourier_bases(self):
        # sample thetas
        self.thetas = torch.randn(
            size=(self.num_of_bases, self.x_dim), 
            device=self.device
        ) / self.lengthscale
        # sample biases
        self.biases = torch.rand(self.num_of_bases, device=self.device) * 2 * pi

    def create_sample(self):
        # sample weights
        self.weights = torch.randn(
            size=(self.num_of_samples, self.num_of_bases), 
            device=self.device
        ).float()

    def calculate_phi(self):
        '''
        From the paper, we are required to calculate a matrix which includes the evaluation of the training set, X_train,
        at the fourier frequencies. This function calculates that matrix, Phi.
        '''
        # we take the dot product by element-wise multiplication followed by summation
        thetas = self.thetas.repeat(self.num_of_train_inputs, 1, 1)
        prod = thetas * self.train_x.unsqueeze(1)
        dot = torch.sum(prod, axis=-1)
        # add biases and take cosine to obtain fourier representations
        ft = torch.cos(dot + self.biases.unsqueeze(0))
        # finally, multiply by corresponding constants (see paper)
        self.Phi = (self.outputscale * np.sqrt(2 / self.num_of_bases) * ft).float()

    def calculate_V(self):
        '''
        From the paper, to give posterior updates we need to calculate the vector V. Since we are doing multiple samples
        at the same time, V will be a matrix. We can pre-calculate it, since its value does not depend on the query locations.
        '''
        # multiply phi matrix by weights
        # PhiW: num_of_train x num_of_samples
        PhiW = torch.matmul(self.Phi, self.weights.T)
        # add noise (see paper)
        PhiW = PhiW + torch.randn(size=PhiW.shape, device=self.device) * self.sigma
        # subtract from training outputs
        mat1 = self.scaled_train_y - PhiW
        # calculate V matrix by premultiplication by inv_mat = (K_nn + I_n*sigma)^{-1}
        # V: num_of_train x num_of_samples
        self.V = torch.matmul(self.inv_mat, mat1)

    def calculate_fourier_features(self, x):
        '''
        Calculate the Fourier Features evaluated at some input x
        '''
        # Make sure x is on the correct device
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, device=self.device)
        else:
            x = x.to(self.device)
        
        # evaluation using fourier features
        self.posterior_update(x)
        # calculate the dot product between the frequencies, theta, and the new query points
        dot = x.matmul(self.thetas.T)
        # calculate the fourier frequency by adding bias and cosine
        ft = torch.cos(dot + self.biases.unsqueeze(0))
        # apply the normalising constants and return the output
        return self.outputscale * np.sqrt(2 / self.num_of_bases) * ft

    def sample_prior(self, x):
        '''
        Create a sample form the prior, evaluate it at x
        '''
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, device=self.device)
        else:
            x = x.to(self.device)
        
        # calculate the fourier features evaluated at the query points
        out1 = self.calculate_fourier_features(x)
        # extend the weights so that we can use element wise multiplication
        weights = self.weights.repeat(self.num_of_multistarts, 1, 1)
        # return the prior
        return torch.sum(weights * out1, axis=-1)

    def posterior_update(self, x):
        '''
        Calculate the posterior update at a location x
        '''
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, device=self.device)
        else:
            x = x.to(self.device)
        
        # x: num_of_multistarts x num_of_samples x dim
        self.calculate_V()  # can probably pre-calculate this
        # train x: num_of_multistarts x num_of_train x dim
        train_x = self.train_x.repeat(self.num_of_multistarts, 1, 1)
        # z: num_of_multistarts x num_of_train x num_of_samples
        # z: kernel evaluation between new query points and training set
        z = self.kernel(train_x, x)
        z = z.evaluate()
        # we now repeat V the number of times necessary so that we can use element-wise multiplication
        V = self.V.repeat(self.num_of_multistarts, 1, 1)
        out = z * V
        return out.sum(axis=1)  # we return the sum across the number of training point, as per the paper

    def query_sample(self, x):
        '''
        Query the sample at a location
        '''
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, device=self.device)
        else:
            x = x.to(self.device)
        
        prior = self.sample_prior(x)
        update = self.posterior_update(x)
        y_scaled = prior + update
        return self.model.outcome_transform.untransform(y_scaled)[0]

    def generate_candidates(self):
        '''
        Generate the Thompson Samples, this function optimizes the samples.
        '''
        # we are always working on [0, 1]^d
        bounds = torch.stack([
            torch.zeros(self.x_dim, device=self.device), 
            torch.ones(self.x_dim, device=self.device)
        ])
        
        # initialise randomly - there is definitely much better ways of doing this
        X = torch.rand(
            self.num_of_multistarts, 
            self.num_of_samples, 
            self.x_dim, 
            device=self.device
        )
        X.requires_grad = True
        
        # define optimiser
        optimiser = torch.optim.Adam([X], lr=self.learning_rate)

        for _ in range(self.num_of_epochs):
            # set zero grad
            optimiser.zero_grad()
            # evaluate loss and backpropagate
            losses = -self.query_sample(X)
            loss = losses.sum()
            loss.backward()
            # take step
            optimiser.step()

            # make sure we are still within the bounds
            for j, (lb, ub) in enumerate(zip(*bounds)):
                X.data[..., j].clamp_(lb, ub)  # need to do this on the data not X itself
        
        # check the final evaluations
        final_evals = self.query_sample(X)
        # choose the best one for each sample
        best_idx = torch.argmax(final_evals, axis=0)
        # return the best one for each sample, without gradients
        X_out = X[best_idx, range(0, self.num_of_samples), :]
        return X_out.detach()

## 2. Download combi data and load it
- Data credits - Richard Liu
- Sample - AlScBN

In [ ]:
def load_data_from_disk(filepath, device='cpu'):
    """
    Load data from disk.
    Expected format: dictionary with 'X_measured' (70, 2) and 'y_measured' (70, 2+)
    """
    print(f"Loading data from {filepath}...")
    # data = torch.load(filepath, map_location=device)
    
    X = data["X_measured"]  # (70, 2)
    Y = data["y_measured"][:, 2]  # (70, 2 or more)
    
    # Convert to tensors if not already
    if not isinstance(X, torch.Tensor):
        X = torch.tensor(X, dtype=torch.float32, device=device)
    if not isinstance(Y, torch.Tensor):
        Y = torch.tensor(Y, dtype=torch.float32, device=device)
    
    # Ensure correct device and dtype
    X = X.to(device).float()
    Y = Y.to(device).float()
    
    # Handle 1D output case
    if Y.dim() == 1:
        Y = Y.unsqueeze(1)
    
    print(f"Loaded X shape: {X.shape}")
    print(f"Loaded Y shape: {Y.shape}")
    
    return X, Y

def visualize_loaded_data(X, Y, save_path='loaded_data_visualization.png'):
    """Visualize the loaded data."""
    # Move to CPU for plotting
    X_cpu = X.cpu().detach().numpy()
    Y_cpu = Y.cpu().detach().numpy()
    
    n_outputs = Y.shape[1]
    n_cols = min(3, n_outputs)
    
    fig, axes = plt.subplots(1, n_cols, figsize=(6*n_cols, 4))
    
    if n_cols == 1:
        axes = [axes]
    
    for i in range(n_cols):
        sc = axes[i].scatter(X_cpu[:, 0], X_cpu[:, 1], c=Y_cpu[:, i], 
                            cmap='viridis', s=60, edgecolors='black', linewidth=0.5)
        axes[i].set_title(f'y_measured[{i}]', fontsize=12, fontweight='bold')
        axes[i].set_xlabel('X position', fontsize=10, fontweight='bold')
        axes[i].set_ylabel('Y position', fontsize=10, fontweight='bold')
        fig.colorbar(sc, ax=axes[i])
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"Data visualization saved to '{save_path}'")

In [ ]:
## download pickle file - gdown--> data credits Richard Liu
import pickle
!gdown https://drive.google.com/uc?id=1SxcpoP-gX5k4FLtb8WjeMRzbmpc6EgK_
pickle_path = "AlScBN_processed_data.pickle"

## 3. Utility functions

### 3a. Reachibility - defines notion of spread

In [ ]:

def reachability_uniformity(behavior, n_bins=25, mins=None, maxs=None, num_filled_grids=100):
    """
    Compute coverage metric based on grid filling in objective space.
    Works for any number of output dimensions.
    """
    filled_grids = set()
    grid_sizes = (maxs - mins) / n_bins
    output_dim = behavior.shape[1]
    
    for point in behavior:
        # Handle boundary cases
        grid_indices = torch.zeros(output_dim, dtype=torch.long, device=behavior.device)
        for d in range(output_dim):
            if point[d] >= maxs[d]:
                grid_indices[d] = n_bins - 1
            else:
                grid_indices[d] = int((point[d] - mins[d]) / grid_sizes[d])
        
        filled_grids.add(tuple(grid_indices.cpu().tolist()))
    
    return len(filled_grids) / num_filled_grids


### 3b. Different acquisiton funcitons for comparison
- Beacon
- MaxVariance
- Novelty search feature space

In [ ]:
class CustomAcquisitionFunction(AcquisitionFunction):
    """BEACON: Novelty search acquisition function using k-NN distance in objective space."""
    
    def __init__(self, model, sampled_behavior, k_NN=10, output_dim=2):
        super().__init__(model=model)
        self.model = model
        self.k_NN = k_NN
        self.sampled_behavior = sampled_behavior
        self.output_dim = output_dim
        
        # Create Thompson samplers for all output dimensions
        self.ts_samplers = []
        for i in range(output_dim):
            sampler = EfficientThompsonSampler(model.models[i])
            sampler.create_sample()
            self.ts_samplers.append(sampler)
    
    @t_batch_mode_transform(expected_q=1)
    def forward(self, X):
        """Compute acquisition function value at X."""
        # Collect samples from all output dimensions
        samples_list = []
        for sampler in self.ts_samplers:
            samples_list.append(sampler.query_sample(X))
        
        samples = torch.cat(samples_list, dim=1)
        
        dist = torch.cdist(samples.to(torch.float64), self.sampled_behavior.to(torch.float64))
        dist, _ = torch.sort(dist, dim=1)
        
        n = dist.size()[1]
        E = torch.cat((torch.ones(self.k_NN, device=dist.device), 
                      torch.zeros(n - self.k_NN, device=dist.device)), dim=0)
        dist = dist * E
        acquisition_values = torch.sum(dist, dim=1)
        
        return acquisition_values.flatten()


class MaxVariance(AcquisitionFunction):
    """MaxVar: Sum of posterior variances across objectives."""
    
    def __init__(
        self,
        model: Model,
        output_dim: int = 2,
        posterior_transform: Optional[PosteriorTransform] = None,
        maximize: bool = True,
    ) -> None:
        super().__init__(model=model)
        self.output_dim = output_dim
    
    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
        """Evaluate the posterior variance on the candidate set X."""
        total_variance = torch.zeros(X.shape[0], device=X.device)
        
        for i in range(self.output_dim):
            posterior = self.model.models[i].posterior(X=X)
            variance = posterior.variance.clamp_min(1e-12)
            total_variance += variance.flatten()
        
        return total_variance


class NoveltySearchFeatureSpace():
    """NS-FS: Novelty search in feature (input) space using k-NN."""
    
    def __init__(self, sampled_X, k=10):
        self.k = k
        self.sampled_X = sampled_X
    
    def __call__(self, X):
        """Compute the acquisition function value at X."""
        dist = torch.cdist(X.to(torch.float64), self.sampled_X.to(torch.float64))
        dist, _ = torch.sort(dist, dim=1)
        n = dist.size()[1]
        E = torch.cat((torch.ones(self.k, device=dist.device), 
                      torch.zeros(n - self.k, device=dist.device)), dim=0)
        dist = dist * E
        acquisition_values = torch.sum(dist, dim=1)
        return acquisition_values.flatten()


## 4. Experiment LOOP

### 4a. run-beacon

In [ ]:
def run_beacon(X_original, Y_original, N_init, BO_iter, n_bins, mins, maxs, 
               num_filled_grids, k_NN, replicate, input_dim, output_dim, device):
    """Run BEACON (Novelty Search in Objective Space with Thompson Sampling)."""
    print("\n" + "="*80)
    print("Running BEACON")
    print("="*80)
    
    cost_tensor = []
    coverage_tensor = []
    
    n_total_points = len(X_original)
    
    for seed in range(replicate):
        print(f'\n=== BEACON Replicate {seed + 1}/{replicate} ===')
        np.random.seed(seed)
        torch.manual_seed(seed)
        
        ids_acquired = np.random.choice(np.arange(n_total_points), size=N_init, replace=False)
        train_x = X_original[ids_acquired].to(device)
        train_y = Y_original[ids_acquired].to(device)
        
        coverage = reachability_uniformity(train_y, n_bins, mins, maxs, num_filled_grids)
        coverage_list = [coverage]
        cost_list = [0]
        
        for i in range(BO_iter):
            if (i + 1) % 20 == 0:
                print(f"  Iteration {i + 1}/{BO_iter}, Coverage: {coverage:.4f}")
            
            # Fit GPs for all output dimensions
            model_list = []
            for obj_idx in range(output_dim):
                covar_module = ScaleKernel(RBFKernel(ard_num_dims=input_dim))
                model_list.append(
                    SingleTaskGP(
                        train_x.to(torch.float64),
                        train_y[:, obj_idx].unsqueeze(1).to(torch.float64),
                        outcome_transform=Standardize(m=1),
                        covar_module=covar_module
                    ).to(device)
                )
            
            model = ModelListGP(*model_list)
            mll = SumMarginalLogLikelihood(model.likelihood, model)
            
            try:
                fit_gpytorch_mll(mll)
            except Exception as e:
                print(f'  Warning: Failed to fit GP: {e}')
            
            # Update training data for all models
            for obj_idx in range(output_dim):
                model.models[obj_idx].train_x = train_x
                model.models[obj_idx].train_y = train_y[:, obj_idx].unsqueeze(1)
            
            # Optimize acquisition
            custom_acq_function = CustomAcquisitionFunction(model, train_y, k_NN=k_NN, output_dim=output_dim)
            acquisition_values = custom_acq_function(X_original.unsqueeze(1).to(torch.float32))
            ids_sorted = acquisition_values.argsort(descending=True)
            
            for id_max in ids_sorted:
                if id_max.item() not in ids_acquired:
                    id_max_acquisition = id_max.item()
                    break
            
            ids_acquired = np.concatenate((ids_acquired, [id_max_acquisition]))
            train_x = X_original[ids_acquired].to(device)
            train_y = Y_original[ids_acquired].to(device)
            
            coverage = reachability_uniformity(train_y, n_bins, mins, maxs, num_filled_grids)
            coverage_list.append(coverage)
            cost_list.append(cost_list[-1] + 1)
        
        cost_tensor.append(cost_list)
        coverage_tensor.append(coverage_list)
        print(f"  Final coverage: {coverage:.4f}")
    
    cost_tensor = torch.tensor(cost_tensor, dtype=torch.float32).cpu()
    coverage_tensor = torch.tensor(coverage_tensor, dtype=torch.float32).cpu()
    
    torch.save(cost_tensor, 'RealData_cost_list_NS.pt')
    torch.save(coverage_tensor, 'RealData_coverage_list_NS.pt')
    print("\nBEACON results saved.")
    
    return cost_tensor, coverage_tensor


### 4b. run-maxvar

In [ ]:

def run_maxvar(X_original, Y_original, N_init, BO_iter, n_bins, mins, maxs, 
               num_filled_grids, replicate, input_dim, output_dim, device):
    """Run MaxVar baseline."""
    print("\n" + "="*80)
    print("Running MaxVar")
    print("="*80)
    
    cost_tensor = []
    coverage_tensor = []
    
    n_total_points = len(X_original)
    
    for seed in range(replicate):
        print(f'\n=== MaxVar Replicate {seed + 1}/{replicate} ===')
        np.random.seed(seed)
        torch.manual_seed(seed)
        
        ids_acquired = np.random.choice(np.arange(n_total_points), size=N_init, replace=False)
        train_x = X_original[ids_acquired].to(device)
        train_y = Y_original[ids_acquired].to(device)
        
        coverage = reachability_uniformity(train_y, n_bins, mins, maxs, num_filled_grids)
        coverage_list = [coverage]
        cost_list = [0]
        
        for i in range(BO_iter):
            if (i + 1) % 20 == 0:
                print(f"  Iteration {i + 1}/{BO_iter}, Coverage: {coverage:.4f}")
            
            # Fit GPs for all output dimensions
            model_list = []
            for obj_idx in range(output_dim):
                covar_module = ScaleKernel(RBFKernel(ard_num_dims=input_dim))
                model_list.append(
                    SingleTaskGP(
                        train_x.to(torch.float64),
                        train_y[:, obj_idx].unsqueeze(1).to(torch.float64),
                        outcome_transform=Standardize(m=1),
                        covar_module=covar_module
                    ).to(device)
                )
            
            model = ModelListGP(*model_list)
            mll = SumMarginalLogLikelihood(model.likelihood, model)
            
            try:
                fit_gpytorch_mll(mll)
            except Exception as e:
                print(f'  Warning: Failed to fit GP: {e}')
            
            # MaxVar acquisition - sum variances across all outputs
            acquisition_values = torch.zeros(len(X_original), device=device)
            for obj_idx in range(output_dim):
                acquisition_values += model.models[obj_idx].posterior(X_original).variance.flatten()
            
            ids_sorted = acquisition_values.argsort(descending=True)
            
            for id_max in ids_sorted:
                if id_max.item() not in ids_acquired:
                    id_max_acquisition = id_max.item()
                    break
            
            ids_acquired = np.concatenate((ids_acquired, [id_max_acquisition]))
            train_x = X_original[ids_acquired].to(device)
            train_y = Y_original[ids_acquired].to(device)
            
            coverage = reachability_uniformity(train_y, n_bins, mins, maxs, num_filled_grids)
            coverage_list.append(coverage)
            cost_list.append(cost_list[-1] + 1)
        
        cost_tensor.append(cost_list)
        coverage_tensor.append(coverage_list)
        print(f"  Final coverage: {coverage:.4f}")
    
    cost_tensor = torch.tensor(cost_tensor, dtype=torch.float32).cpu()
    coverage_tensor = torch.tensor(coverage_tensor, dtype=torch.float32).cpu()
    
    torch.save(cost_tensor, 'RealData_cost_list_MaxVar.pt')
    torch.save(coverage_tensor, 'RealData_coverage_list_MaxVar.pt')
    print("\nMaxVar results saved.")
    
    return cost_tensor, coverage_tensor


### 4c. run-Random-search

In [ ]:
def run_random_search(X_original, Y_original, N_init, BO_iter, n_bins, mins, maxs, 
                      num_filled_grids, replicate, input_dim, output_dim, device):
    """Run Random Search baseline."""
    print("\n" + "="*80)
    print("Running Random Search")
    print("="*80)
    
    cost_tensor = []
    coverage_tensor = []
    
    n_total_points = len(X_original)
    
    for seed in range(replicate):
        print(f'\n=== Random Search Replicate {seed + 1}/{replicate} ===')
        np.random.seed(seed)
        torch.manual_seed(seed)
        
        ids_acquired = np.random.choice(np.arange(n_total_points), size=N_init, replace=False)
        train_x = X_original[ids_acquired].to(device)
        train_y = Y_original[ids_acquired].to(device)
        
        coverage = reachability_uniformity(train_y, n_bins, mins, maxs, num_filled_grids)
        coverage_list = [coverage]
        cost_list = [0]
        
        for i in range(BO_iter):
            if (i + 1) % 20 == 0:
                print(f"  Iteration {i + 1}/{BO_iter}, Coverage: {coverage:.4f}")
            
            # Random acquisition
            acquisition_values = torch.rand(n_total_points, device=device)
            ids_sorted = acquisition_values.argsort(descending=True)
            
            for id_max in ids_sorted:
                if id_max.item() not in ids_acquired:
                    id_max_acquisition = id_max.item()
                    break
            
            ids_acquired = np.concatenate((ids_acquired, [id_max_acquisition]))
            train_x = X_original[ids_acquired].to(device)
            train_y = Y_original[ids_acquired].to(device)
            
            coverage = reachability_uniformity(train_y, n_bins, mins, maxs, num_filled_grids)
            coverage_list.append(coverage)
            cost_list.append(cost_list[-1] + 1)
        
        cost_tensor.append(cost_list)
        coverage_tensor.append(coverage_list)
        print(f"  Final coverage: {coverage:.4f}")
    
    cost_tensor = torch.tensor(cost_tensor, dtype=torch.float32).cpu()
    coverage_tensor = torch.tensor(coverage_tensor, dtype=torch.float32).cpu()
    
    torch.save(cost_tensor, 'RealData_cost_list_RS.pt')
    torch.save(coverage_tensor, 'RealData_coverage_list_RS.pt')
    print("\nRandom Search results saved.")
    
    return cost_tensor, coverage_tensor

### 4d. run-novelty-search-feature-space

In [ ]:
def run_ns_feature_space(X_original, Y_original, N_init, BO_iter, n_bins, mins, maxs, 
                         num_filled_grids, k_NN, replicate, input_dim, output_dim, device):
    """Run Novelty Search in Feature Space baseline."""
    print("\n" + "="*80)
    print("Running NS-FS (Novelty Search in Feature Space)")
    print("="*80)
    
    cost_tensor = []
    coverage_tensor = []
    
    n_total_points = len(X_original)
    
    for seed in range(replicate):
        print(f'\n=== NS-FS Replicate {seed + 1}/{replicate} ===')
        np.random.seed(seed)
        torch.manual_seed(seed)
        
        ids_acquired = np.random.choice(np.arange(n_total_points), size=N_init, replace=False)
        train_x = X_original[ids_acquired].to(device)
        train_y = Y_original[ids_acquired].to(device)
        
        coverage = reachability_uniformity(train_y, n_bins, mins, maxs, num_filled_grids)
        coverage_list = [coverage]
        cost_list = [0]
        
        for i in range(BO_iter):
            if (i + 1) % 20 == 0:
                print(f"  Iteration {i + 1}/{BO_iter}, Coverage: {coverage:.4f}")
            
            # NS in feature space
            custom_acq_function = NoveltySearchFeatureSpace(train_x, k=k_NN)
            acquisition_values = custom_acq_function(X_original)
            ids_sorted = acquisition_values.argsort(descending=True)
            
            for id_max in ids_sorted:
                if id_max.item() not in ids_acquired:
                    id_max_acquisition = id_max.item()
                    break
            
            ids_acquired = np.concatenate((ids_acquired, [id_max_acquisition]))
            train_x = X_original[ids_acquired].to(device)
            train_y = Y_original[ids_acquired].to(device)
            
            coverage = reachability_uniformity(train_y, n_bins, mins, maxs, num_filled_grids)
            coverage_list.append(coverage)
            cost_list.append(cost_list[-1] + 1)
        
        cost_tensor.append(cost_list)
        coverage_tensor.append(coverage_list)
        print(f"  Final coverage: {coverage:.4f}")
    
    cost_tensor = torch.tensor(cost_tensor, dtype=torch.float32).cpu()
    coverage_tensor = torch.tensor(coverage_tensor, dtype=torch.float32).cpu()
    
    torch.save(cost_tensor, 'RealData_cost_list_NS_xspace.pt')
    torch.save(coverage_tensor, 'RealData_coverage_list_NS_xspace.pt')
    print("\nNS-FS results saved.")
    
    return cost_tensor, coverage_tensor

## 5. Main function to run all 

In [ ]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("="*80)
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory Allocated: {torch.cuda.memory_allocated(0)/1024**2:.2f} MB")
print("="*80)

# Load data from disk
print("\nLOADING DATA FROM DISK")
print("="*80)
data_filepath = 'your_data_file.pt'  # CHANGE THIS to your actual file path
X_original, Y_original = load_data_from_disk(data_filepath, device=device)

# Automatically detect dimensions
input_dim = X_original.shape[1]
output_dim = Y_original.shape[1]
n_total_points = len(X_original)

print(f"\n{'='*80}")
print(f"DATASET INFORMATION")
print(f"{'='*80}")
print(f"Total points: {n_total_points}")
print(f"Input dimensions: {input_dim}")
print(f"Output dimensions: {output_dim}")

# Visualize loaded data
visualize_loaded_data(X_original, Y_original)

# Hyperparameters - adjust based on dataset size
N_init = max(5, n_total_points // 10)  # 10% of data or at least 5 points
max_BO_iter = n_total_points - N_init - 1  # Maximum possible iterations
BO_iter = min(50, max_BO_iter)  # Use 50 or max available
replicate = 5
n_bins = 10
k_NN = max(3, min(10, N_init // 2))  # Adjust k_NN based on init size

print(f"\nHYPERPARAMETERS")
print(f"{'='*80}")
print(f"Initial samples (N_init): {N_init}")
print(f"BO iterations: {BO_iter}")
print(f"Number of replicates: {replicate}")
print(f"Grid bins: {n_bins}")
print(f"k-NN: {k_NN}")

# Compute bounds for coverage metric
mins = torch.min(Y_original, dim=0).values
maxs = torch.max(Y_original, dim=0).values
print(f"\nOUTPUT SPACE BOUNDS")
print(f"{'='*80}")
for i in range(output_dim):
    print(f"Output {i}: [{mins[i]:.4f}, {maxs[i]:.4f}]")

# Calculate filled grids
grid_sizes = (maxs - mins) / n_bins
filled_grids = set()
for point in Y_original:
    grid_indices = torch.zeros(output_dim, dtype=torch.long, device=device)
    for d in range(output_dim):
        if point[d] >= maxs[d]:
            grid_indices[d] = n_bins - 1
        else:
            grid_indices[d] = int((point[d] - mins[d]) / grid_sizes[d])
    filled_grids.add(tuple(grid_indices.cpu().tolist()))

num_filled_grids = len(filled_grids)
max_possible_grids = n_bins ** output_dim
print(f"\nGRID COVERAGE")
print(f"{'='*80}")
print(f"Total unique grids filled: {num_filled_grids} out of {max_possible_grids}")
print(f"Initial coverage: {num_filled_grids/max_possible_grids*100:.2f}%")

# Run all methods
cost_NS, coverage_NS = run_beacon(X_original, Y_original, N_init, BO_iter, n_bins, 
                                    mins, maxs, num_filled_grids, k_NN, replicate, 
                                    input_dim, output_dim, device)

cost_MaxVar, coverage_MaxVar = run_maxvar(X_original, Y_original, N_init, BO_iter, 
                                            n_bins, mins, maxs, num_filled_grids, replicate, 
                                            input_dim, output_dim, device)

cost_RS, coverage_RS = run_random_search(X_original, Y_original, N_init, BO_iter, 
                                            n_bins, mins, maxs, num_filled_grids, replicate, 
                                            input_dim, output_dim, device)

cost_NS_xspace, coverage_NS_xspace = run_ns_feature_space(X_original, Y_original, N_init, 
                                                            BO_iter, n_bins, mins, maxs, 
                                                            num_filled_grids, k_NN, replicate, 
                                                            input_dim, output_dim, device)
print("\n" + "="*80)
print("GENERATING COMPARISON PLOTS")
print("="*80)

# Compute statistics
coverage_NS_mean = torch.mean(coverage_NS, dim=0)
coverage_NS_std = torch.std(coverage_NS, dim=0)
cost_NS_mean = torch.mean(cost_NS, dim=0)

coverage_MaxVar_mean = torch.mean(coverage_MaxVar, dim=0)
coverage_MaxVar_std = torch.std(coverage_MaxVar, dim=0)
cost_MaxVar_mean = torch.mean(cost_MaxVar, dim=0)

coverage_RS_mean = torch.mean(coverage_RS, dim=0)
coverage_RS_std = torch.std(coverage_RS, dim=0)
cost_RS_mean = torch.mean(cost_RS, dim=0)

coverage_NS_xspace_mean = torch.mean(coverage_NS_xspace, dim=0)
coverage_NS_xspace_std = torch.std(coverage_NS_xspace, dim=0)
cost_NS_xspace_mean = torch.mean(cost_NS_xspace, dim=0)

# Plot
text_size = 24
marker_size = 18
weight = 'bold'
alpha = 0.3
linewidth = 4
marker_interval = max(1, BO_iter // 10)  # Adaptive marker interval

plt.figure(figsize=(12, 10))
plt.plot(cost_NS_mean[::marker_interval], coverage_NS_mean[::marker_interval], 
            label='BEACON', marker='X', markersize=marker_size, linewidth=linewidth)
plt.plot(cost_MaxVar_mean[::marker_interval], coverage_MaxVar_mean[::marker_interval], 
            label='MaxVar', marker='^', markersize=marker_size, linewidth=linewidth)
plt.plot(cost_NS_xspace_mean[::marker_interval], coverage_NS_xspace_mean[::marker_interval], 
            label='NS-FS', marker='p', markersize=marker_size, color='mediumpurple', linewidth=linewidth)
plt.plot(cost_RS_mean[::marker_interval], coverage_RS_mean[::marker_interval], 
            label='RS', marker='v', markersize=marker_size, color='hotpink', linewidth=linewidth)

plt.fill_between(cost_NS_mean, coverage_NS_mean - coverage_NS_std, 
                    coverage_NS_mean + coverage_NS_std, alpha=alpha)
plt.fill_between(cost_MaxVar_mean, coverage_MaxVar_mean - coverage_MaxVar_std, 
                    coverage_MaxVar_mean + coverage_MaxVar_std, alpha=alpha)
plt.fill_between(cost_NS_xspace_mean, coverage_NS_xspace_mean - coverage_NS_xspace_std, 
                    coverage_NS_xspace_mean + coverage_NS_xspace_std, alpha=alpha, color='mediumpurple')
plt.fill_between(cost_RS_mean, coverage_RS_mean - coverage_RS_std, 
                    coverage_RS_mean + coverage_RS_std, alpha=alpha, color='hotpink')

plt.xlabel('Number of evaluations', fontsize=text_size, fontweight=weight)
plt.ylabel('Reachability', fontsize=text_size, fontweight=weight)
plt.legend(prop={'weight': 'bold', 'size': text_size}, loc='best')
plt.title(f'Real Data ({n_total_points} points, input_dim={input_dim}, output_dim={output_dim})', 
            fontsize=text_size, fontweight=weight)

plt.tick_params(axis='both', which='both', width=2)
ax = plt.gca()
for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontsize(text_size)
    label.set_fontweight('bold')

for spine in ax.spines.values():
    spine.set_linewidth(2)

plt.grid(alpha=0.5, linewidth=2.0)
plt.tight_layout()
plt.savefig("comparison_all_methods_realdata.png", dpi=150, bbox_inches='tight')

# Print GPU memory usage if applicable
if device.type == 'cuda':
    print(f"\nFinal GPU Memory Allocated: {torch.cuda.memory_allocated(0)/1024**2:.2f} MB")
    print(f"Peak GPU Memory Allocated: {torch.cuda.max_memory_allocated(0)/1024**2:.2f} MB")

print("\n" + "="*80)
print("EXPERIMENT COMPLETE!")
print("="*80)
